In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
###for dirname, _, filenames in os.walk('/kaggle/input'):
###    for filename in filenames:
###        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Initialize Environment

In [ ]:
# !pip install -q efficientnet >> /dev/null

In [ ]:
import pandas as pd, numpy as np
from kaggle_datasets import KaggleDatasets
import tensorflow as tf, re, math
import tensorflow.keras.backend as K
#import efficientnet.tfkeras as efn
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

import cv2

## Configuration
In order to be a proper cross validation with a meaningful overall CV score (aligned with LB score), **you need to choose the same** `IMG_SIZES`, `INC2019`, `INC2018`, and `EFF_NETS` **for each fold**. If your goal is to just run lots of experiments, then you can choose to have a different experiment in each fold. Then each fold is like a holdout validation experiment. When you find a configuration you like, you can use that configuration for all folds. 
* DEVICE - is GPU or TPU
* SEED - a different seed produces a different triple stratified kfold split.
* FOLDS - number of folds. Best set to 3, 5, or 15 but can be any number between 2 and 15
* IMG_SIZES - is a Python list of length FOLDS. These are the image sizes to use each fold
* INC2019 - This includes the new half of the 2019 competition data. The second half of the 2019 data is the comp data from 2018 plus 2017
* INC2018 - This includes the second half of the 2019 competition data which is the comp data from 2018 plus 2017
* BATCH_SIZES - is a list of length FOLDS. These are batch sizes for each fold. For maximum speed, it is best to use the largest batch size your GPU or TPU allows.
* EPOCHS - is a list of length FOLDS. These are maximum epochs. Note that each fold, the best epoch model is saved and used. So if epochs is too large, it won't matter.
* EFF_NETS - is a list of length FOLDS. These are the EfficientNets to use each fold. The number refers to the B. So a number of `0` refers to EfficientNetB0, and `1` refers to EfficientNetB1, etc.
* WGTS - this should be `1/FOLDS` for each fold. This is the weight when ensembling the folds to predict the test set. If you want a weird ensemble, you can use different weights.
* TTA - test time augmentation. Each test image is randomly augmented and predicted TTA times and the average prediction is used. TTA is also applied to OOF during validation.

In [ ]:
DEVICE = "TPU" #or "GPU" # 

# USE DIFFERENT SEED FOR DIFFERENT STRATIFIED KFOLD
SEED = 42

# NUMBER OF FOLDS. USE 3, 5, OR 15 
FOLDS = 3

# WHICH IMAGE SIZES TO LOAD EACH FOLD
# CHOOSE 128, 192, 256, 384, 512, 768, 1024, 1536 
#IMG_SIZES = [384,384,384,384,384]
#IMG_SIZES = [128,128,128,128,128]
SIZE = 1024
IMG_SIZES = [SIZE]*FOLDS

# INCLUDE OLD COMP DATA? YES=1 NO=0
#TOKEN = 0
#INC2019 = [TOKEN,TOKEN,TOKEN,TOKEN,TOKEN]
#INC2018 = [TOKEN,TOKEN,TOKEN,TOKEN,TOKEN]

# BATCH SIZE AND EPOCHS
# Original batch size 
#BATCH_SIZES = [32]*FOLDS

# YG 
# Experimental batch sizes for EfficientNetB7
# BAD
#BATCH_SIZES = [4]*FOLDS # for 768 - VERY LONG - stops after 2 folds only, ~4 hours per fold
#BATCH_SIZES = [8]*FOLDS # for 768 - VERY LONG - 3 hours per fold
#BATCH_SIZES = [12]*FOLDS # for 768 - BAD, not enough resources 
# GOOD
#BATCH_SIZES = [32]*FOLDS # for 384 - GOOD - ~1 hour per fold
#BATCH_SIZES = [16]*FOLDS # for 512

# GPU: Experimental batch sizes for EfficientNetB0
# BATCH_SIZES = [2]*FOLDS # SIZE = 1024

# Experimental batch sizes for EfficientNetB1
BATCH = 1
BATCH_SIZES = [BATCH]*FOLDS # SIZE = 768

# YG 
# NASNetLarge
# BAD
#BATCH_SIZES = [4]*FOLDS # for 768 - VERY LONG - ~3.5-4 hours per fold 
#BATCH_SIZES = [8]*FOLDS # for 768 - BAD - not enough resources
# GOOD
#BATCH_SIZES = [16]*FOLDS # for 384 - GOOD - ~1 hour per fold
#BATCH_SIZES = [8]*FOLDS # for 512

#EPOCHS = [12]*FOLDS
EPOCH = 32
EPOCHS = [EPOCH]*FOLDS

# EfficientNet
# WHICH EFFICIENTNET B? TO USE
model_name = 'EfficientNet'
#MODEL = 7 # EfficientNetB7
MODEL = 0 # EfficientNetB3
#MODEL = 0 # EfficientNetB0
EFF_NETS = [MODEL]*FOLDS
SPECIFIC_SIZE = SIZE

# MobileNetV2
#MODEL = 'MobileNetV2'
#SPECIFIC_SIZE = SIZE

#model_name = 'NASNetMobile'
#MODEL = 'NASNetMobile'
#SPECIFIC_SIZE = SIZE # 224 only at the moment!

#model_name = 'NASNetLarge'
#MODEL = 'NASNetLarge'
#SPECIFIC_SIZE = SIZE # 331 only at the moment!

# WEIGHTS FOR FOLD MODELS WHEN PREDICTING TEST
WGTS = [1/FOLDS]*FOLDS

# TEST TIME AUGMENTATION STEPS
TTA = 11

# make the weights and biases of the base model non-trainable
# by "freezing" each layer of the BASE network
TRAINABLE = True

AUGMENTATION = True

AUG_LOSSLESS = False

In [ ]:
# Time estimation - 1 epoch

# Model = EfficientNetB0
# MODEL = 0

########################################################
# Image Size
# SIZE = 1536
# BATCH = 1

# Fold Times:
# FOLDS = 3
# 1it [03:29, 209.24s/it]
# 2it [06:58, 209.25s/it]
# 3it [10:25, 208.61s/it]
# CPU times: user 2min 11s, sys: 13.6 s, total: 2min 25s
# Wall time: 10min 25s

# + 5 min for testing

########################################################
# Image Size
# SIZE = 1024
# BATCH = 8

# CPU times: user 2min 18s, sys: 13.8 s, total: 2min 32s
# Wall time: 6min 57s

########################################################


########################################################
# Image Size
# SIZE = 768
# BATCH = 16

# CPU times: user 2min 15s, sys: 12.9 s, total: 2min 28s
# Wall time: 6min 20s


In [ ]:
# Time estimation - 1 epoch

# Model = EfficientNetB7
# MODEL = 7

######################################################## MIN
# Image Size
# SIZE = 128
# BATCH = 32

# CPU times: user 7min 4s, sys: 46.5 s, total: 7min 50s
# Wall time: 13min 25s

######################################################## MAX
# Image Size
# SIZE = 1024
# BATCH = 1

# CPU times: user 7min 19s, sys: 47.8 s, total: 8min 6s
# Wall time: 18min 27s

########################################################
# Image Size
# SIZE = 768
# BATCH = 4

# CPU times: user 7min 27s, sys: 49.2 s, total: 8min 16s
# Wall time: 17min 40s

########################################################
# Image Size
# SIZE = 512
# BATCH = 16

# CPU times: user 7min 11s, sys: 47.5 s, total: 7min 58s
# Wall time: 15min 50s

########################################################
# Image Size
# SIZE = 256
# BATCH = 16

# CPU times: user 4min 43s, sys: 40.4 s, total: 5min 24s
# Wall time: 7min 9s

########################################################
# TRAINABLE = False
########################################################
# Image Size
# SIZE = 256
# BATCH = 16

# CPU times: user 4min 51s, sys: 41.1 s, total: 5min 32s
# Wall time: 7min 12s

########################################################
# TRAINABLE = False
# AUGMENTATION = False
########################################################
# Image Size
# SIZE = 256
# BATCH = 16

# CPU times: user 4min 35s, sys: 38.1 s, total: 5min 13s
# Wall time: 6min 56s

In [ ]:
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

# Step 1: Preprocess
Preprocess has already been done and saved to TFRecords. Here we choose which size to load. We can use either 128x128, 192x192, 256x256, 384x384, 512x512, 768x768 by changing the `IMG_SIZES` variable in the preceeding code section. These TFRecords are discussed [here][1]. The advantage of using different input sizes is discussed [here][2]

[1]: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/155579
[2]: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/160147

In [ ]:
! ls ../input/lyme-disease-rashes-in-tfrecords

In [ ]:
print(KaggleDatasets().get_gcs_path('lyme-disease-rashes-in-tfrecords'))

In [ ]:
%%time

GCS_PATH = [None]*FOLDS 
#GCS_PATH2 = [None]*FOLDS
for i,k in enumerate(IMG_SIZES):
    #GCS_PATH[i] = KaggleDatasets().get_gcs_path('melanoma-%ix%i'%(k,k))
    #GCS_PATH2[i] = KaggleDatasets().get_gcs_path('isic2019-%ix%i'%(k,k))
    GCS_PATH[i] = KaggleDatasets().get_gcs_path('lyme-disease-rashes-in-tfrecords')

files_train = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[0] + '/train_%d_*.tfrec'%SIZE)))
files_test  = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[0] + '/test_%d_*.tfrec'%SIZE)))

In [ ]:
files_train

In [ ]:
files_test

# Step 2: Data Augmentation
This notebook uses rotation, sheer, zoom, shift augmentation first shown in this notebook [here][1] and successfully used in Melanoma comp by AgentAuers [here][2]. This notebook also uses horizontal flip, hue, saturation, contrast, brightness augmentation similar to last years winner and also similar to AgentAuers' notebook.

Additionally we can decide to use external data by changing the variables `INC2019` and `INC2018` in the preceeding code section. These variables respectively indicate whether to load last year 2019 data and/or year 2018 + 2017 data. These datasets are discussed [here][3]

Consider experimenting with different augmenation and/or external data. The code to load TFRecords is taken from AgentAuers' notebook [here][2]. Thank you AgentAuers, this is great work.

[1]: https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96
[2]: https://www.kaggle.com/agentauers/incredible-tpus-finetune-effnetb0-b6-at-once
[3]: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/164910

In [ ]:
ROT_ = 180.0
SHR_ = 2.0
HZOOM_ = 8.0
WZOOM_ = 8.0
HSHIFT_ = 8.0
WSHIFT_ = 8.0

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear    = math.pi * shear    / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
    c1   = tf.math.cos(rotation)
    s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
    rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
                                   -s1,  c1,   zero, 
                                   zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                                zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), 
                 K.dot(zoom_matrix,     shift_matrix))


def transform(image, DIM=256):    
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    XDIM = DIM%2 #fix for size 331
    
    rot = ROT_ * tf.random.normal([1], dtype='float32')
    shr = SHR_ * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM_
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM_
    h_shift = HSHIFT_ * tf.random.normal([1], dtype='float32') 
    w_shift = WSHIFT_ * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x   = tf.repeat(tf.range(DIM//2, -DIM//2,-1), DIM)
    y   = tf.tile(tf.range(-DIM//2, DIM//2), [DIM])
    z   = tf.ones([DIM*DIM], dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -DIM//2+XDIM+1, DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack([DIM//2-idx2[0,], DIM//2-1+idx2[1,]])
    d    = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM, DIM,3])

In [ ]:
def read_labeled_tfrecord(example):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'image_name'                   : tf.io.FixedLenFeature([], tf.string),
        #'patient_id'                   : tf.io.FixedLenFeature([], tf.int64),
        #'sex'                          : tf.io.FixedLenFeature([], tf.int64),
        #'age_approx'                   : tf.io.FixedLenFeature([], tf.int64),
        #'anatom_site_general_challenge': tf.io.FixedLenFeature([], tf.int64),
        #'diagnosis'                    : tf.io.FixedLenFeature([], tf.int64),
        'target'                       : tf.io.FixedLenFeature([], tf.int64)
    }           
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], example['target']


def read_unlabeled_tfrecord(example, return_image_name):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'image_name'                   : tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], example['image_name'] if return_image_name else 0

 
def prepare_image(img, augment=True, aug_lossless=True, dim=256):    
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    
    if augment:
        if aug_lossless:
            img = transform(img,DIM=dim)
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            img = tf.image.rot90(img, tf.random.uniform(shape=[], minval=0, maxval=3, dtype=tf.int32))
        else:
            img = transform(img,DIM=dim)
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            img = tf.image.rot90(img, tf.random.uniform(shape=[], minval=0, maxval=3, dtype=tf.int32))
           
            img = tf.image.random_hue(img, 0.01)
            img = tf.image.random_saturation(img, 0.7, 1.3)
            img = tf.image.random_contrast(img, 0.8, 1.2)
            img = tf.image.random_brightness(img, 0.1)
            img = tf.image.random_jpeg_quality(img, 75, 95)

    # YG - just for adaptation to NASNet ... and other families
#    else:
#        if(MODEL == 'NASNetMobile' or MODEL == 'NASNetLarge'):
#            img = transform(img,DIM=dim)    
    
    #img = cv2.resize(img, (dim, dim),interpolation = cv2.INTER_CUBIC)
                              
    img = tf.reshape(img, [dim,dim, 3])
    #img = tf.image.resize(img, [224, 224, 3])
            
    return img

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) 
         for filename in filenames]
    return np.sum(n)

In [ ]:
def get_dataset(files, augment = False, aug_lossless = True, shuffle = False, repeat = False, 
                labeled=True, return_image_names=True, batch_size=16, dim=256):
    
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    ds = ds.cache()
    
    if repeat:
        ds = ds.repeat()
    
    if shuffle: 
        ds = ds.shuffle(1024*8)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
        
    if labeled: 
        ds = ds.map(read_labeled_tfrecord, num_parallel_calls=AUTO)
    else:
        ds = ds.map(lambda example: read_unlabeled_tfrecord(example, return_image_names), 
                    num_parallel_calls=AUTO)      
    
    ds = ds.map(lambda img, imgname_or_label: (prepare_image(img, augment=augment, aug_lossless = aug_lossless, dim=dim), 
                                               imgname_or_label), 
                num_parallel_calls=AUTO)
    
    ds = ds.batch(batch_size * REPLICAS)
    ds = ds.prefetch(AUTO)
    return ds

# Step 3: Build Model
This is a common model architecute. Consider experimenting with different backbones, custom heads, losses, and optimizers. Also consider inputing meta features into your CNN.

In [ ]:
# YG
# These models HAVE PASSED the initial test!

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.applications import DenseNet121

from tensorflow.keras.applications import EfficientNetB0

dim=SPECIFIC_SIZE

#if(model_name =='EfficientNet'):

# These should be checked:
#from tensorflow.keras.applications import InceptionV3
#model_name = 'InceptionV3'
#from tensorflow.keras.applications import InceptionResNetV2
#model_name = 'InceptionResNetV2'
#from tensorflow.keras.applications import MobileNetV2
#model_name = 'MobileNetV2'
#from tensorflow.keras.applications import ResNet101
#model_name = 'ResNet101'
#from tensorflow.keras.applications import ResNet101V2
#model_name = 'ResNet101V2'
#from tensorflow.keras.applications import VGG16
#model_name = 'VGG16'
#from tensorflow.keras.applications import Xception
#model_name = 'Xception'



def build_model(model_name=model_name, dim=SPECIFIC_SIZE, trainable=False):
    inp = tf.keras.layers.Input(shape=(dim,dim,3))
    
    if(model_name =='EfficientNet'):
#        EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
#            efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]
#        base = EFNS[MODEL](input_shape=(dim,dim,3),weights='imagenet',include_top=False)
        if(MODEL==0):
            base = EfficientNetB0(input_shape=(dim,dim,3),weights='imagenet',include_top=False)
            model_name = 'EfficientNetB' + str(MODEL)
        else:
            print('ERROR: This code is prepared for EfficientNetB, version' + str(MODEL) + ', BUT other version is used!')
        
    if(model_name =='MobileNetV2'):
        base = MobileNetV2(input_shape=(dim,dim,3),weights='imagenet',include_top=False)
    
    if(model_name =='NASNetMobile'):
        base = NASNetMobile(input_shape=(dim,dim,3),weights='imagenet',include_top=False)

    if(model_name =='NASNetLarge'):    
        base = NASNetLarge(input_shape=(dim,dim,3),weights='imagenet',include_top=False)    

    if(model_name == 'DenseNet121'):
        base = DenseNet121(input_shape=(dim,dim,3),weights='imagenet',include_top=False)       
    
    # make the weights and biases of the base model non-trainable
    # by "freezing" each layer of the BASE network
    for layer in base.layers:
        layer.trainable = trainable

    x = base(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1,activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inp,outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05) 
    model.compile(optimizer=opt,loss=loss,metrics=['AUC'])
    return model

# Step 4: Train Schedule
This is a common train schedule for transfer learning. The learning rate starts near zero, then increases to a maximum, then decays over time. Consider changing the schedule and/or learning rates. Note how the learning rate max is larger with larger batches sizes. This is a good practice to follow.

In [ ]:
def get_lr_callback(batch_size=8):
    lr_start   = 0.000005
    lr_max     = 0.00000125 * REPLICAS * batch_size
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

## Train Model
Our model will be trained for the number of FOLDS and EPOCHS you chose in the configuration above. Each fold the model with lowest validation loss will be saved and used to predict OOF and test. Adjust the variables `VERBOSE` and `DISPLOY_PLOT` below to determine what output you want displayed. The variable `VERBOSE=1 or 2` will display the training and validation loss and auc for each epoch as text. The variable `DISPLAY_PLOT` shows this information as a plot. 

In [ ]:
from tqdm import tqdm

In [ ]:
! nvidia-smi

In [ ]:
%%time
# USE VERBOSE=0 for silent, VERBOSE=1 for interactive, VERBOSE=2 for commit
VERBOSE = 1
DISPLAY_PLOT = True

skf = KFold(n_splits=FOLDS,shuffle=True,random_state=SEED)
oof_pred = []; oof_tar = []; oof_val = []; oof_names = []; oof_folds = [] 
preds = np.zeros((count_data_items(files_test),1))

for fold,(idxT,idxV) in tqdm(enumerate(skf.split(np.arange(14)))):
    
    # DISPLAY FOLD INFO
    if DEVICE=='TPU':
        if tpu: tf.tpu.experimental.initialize_tpu_system(tpu)
    print('#'*25); print('#### FOLD',fold+1)
# YG    
#    print('#### Image Size %i with EfficientNet B%i and batch_size %i'%
#          (IMG_SIZES[fold],EFF_NETS[fold],BATCH_SIZES[fold]*REPLICAS))
    print('#### Image Size %i with %s and batch_size %i'%
          (IMG_SIZES[fold],model_name,BATCH_SIZES[fold]*REPLICAS))
    
    # CREATE TRAIN AND VALIDATION SUBSETS
    #files_train = tf.io.gfile.glob([GCS_PATH[fold] + '/train%.2i*.tfrec'%x for x in idxT])
    print('SIZE:',SIZE)
    filename_core = GCS_PATH[fold] + '/train_%i_'%(SIZE)
    files_train = tf.io.gfile.glob([filename_core + '%.2i*.tfrec'%x for x in idxT])
    #print([GCS_PATH[fold]])
    #print([GCS_PATH[fold] + '/train_128_%.2i*.tfrec'%x for x in idxT])
    print('Files for TRAIN:')
    print(files_train)

#    if INC2019[fold]:
#        files_train += tf.io.gfile.glob([GCS_PATH2[fold] + '/train%.2i*.tfrec'%x for x in idxT*2+1])
#        print('#### Using 2019 external data')
#    if INC2018[fold]:
#        files_train += tf.io.gfile.glob([GCS_PATH2[fold] + '/train%.2i*.tfrec'%x for x in idxT*2])
#        print('#### Using 2018+2017 external data')
        
    np.random.shuffle(files_train); print('#'*25)
    
    filename_core = GCS_PATH[fold] + '/train_%i_'%(SIZE)
    files_valid = tf.io.gfile.glob([filename_core + '%.2i*.tfrec'%x for x in idxV])
    print('Files for VALIDATION:')
    print(files_valid)
    
    files_test = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[fold] + '/test_%i_*.tfrec'%(SIZE))))
    print('Files for TEST:')
    print(files_test)
    
    # BUILD MODEL
    K.clear_session()
    with strategy.scope():
        model = build_model(model_name=model_name, dim=SPECIFIC_SIZE, trainable=TRAINABLE)   
        model.summary() # YG
        
    # SAVE BEST MODEL EACH FOLD
    sv = tf.keras.callbacks.ModelCheckpoint(
        DEVICE + '_' + str(MODEL) + '_' + str(SIZE) + '_fold-%i.h5'%fold, monitor='val_loss', verbose=0, save_best_only=True,
        save_weights_only=True, mode='min', save_freq='epoch')
   
    # TRAIN
    print('Training...')
    history = model.fit(
        get_dataset(files_train, augment=True, aug_lossless = AUG_LOSSLESS, shuffle=True, repeat=True,
                    #dim=IMG_SIZES[fold], 
                    dim=SPECIFIC_SIZE, batch_size = BATCH_SIZES[fold]), 
        epochs=EPOCHS[fold], callbacks = [sv,get_lr_callback(BATCH_SIZES[fold])], 
        steps_per_epoch=count_data_items(files_train)/BATCH_SIZES[fold]//REPLICAS,
        validation_data=get_dataset(files_valid,augment=False,shuffle=False,
                repeat=False,
                #dim=IMG_SIZES[fold]
                dim=SPECIFIC_SIZE), #class_weight = {0:1,1:2},
        verbose=VERBOSE
    )
    
    print('Loading best model...')
    model.load_weights(DEVICE + '_' + str(MODEL) + '_' + str(SIZE) + '_fold-%i.h5'%fold)
    
    # PREDICT OOF USING TTA
    print('Predicting OOF with TTA...')
    ds_valid = get_dataset(files_valid,labeled=False, return_image_names=False,
                           augment=AUGMENTATION, aug_lossless = AUG_LOSSLESS, 
            repeat=True,shuffle=False,dim=SPECIFIC_SIZE,                        
            #dim=IMG_SIZES[fold],
            batch_size=BATCH_SIZES[fold]*4)
    ct_valid = count_data_items(files_valid); STEPS = TTA * ct_valid/BATCH_SIZES[fold]/4/REPLICAS
    pred = model.predict(ds_valid,steps=STEPS,verbose=VERBOSE)[:TTA*ct_valid,] 
    oof_pred.append( np.mean(pred.reshape((ct_valid,TTA),order='F'),axis=1) )                 
    #oof_pred.append(model.predict(get_dataset(files_valid,dim=IMG_SIZES[fold]),verbose=1))
    
    # GET OOF TARGETS AND NAMES
    ds_valid = get_dataset(files_valid, 
                           augment=False, aug_lossless = False, 
                           repeat=False, dim=SPECIFIC_SIZE,                        
            #dim=IMG_SIZES[fold],
            labeled=True, return_image_names=True)
    oof_tar.append( np.array([target.numpy() for img, target in iter(ds_valid.unbatch())]) )
    oof_folds.append( np.ones_like(oof_tar[-1],dtype='int8')*fold )
    ds = get_dataset(files_valid, 
                     augment=False, aug_lossless = False, 
                     repeat=False, dim=SPECIFIC_SIZE,
                     #dim=IMG_SIZES[fold],
                labeled=False, return_image_names=True)
    oof_names.append( np.array([img_name.numpy().decode("utf-8") for img, img_name in iter(ds.unbatch())]))
    
    # PREDICT TEST USING TTA
    print('Predicting Test with TTA...')
    ds_test = get_dataset(files_test,labeled=False,return_image_names=False,
                          augment=True, aug_lossless = AUG_LOSSLESS, 
            repeat=True,shuffle=False,dim=SPECIFIC_SIZE,                        
            #dim=IMG_SIZES[fold],
            batch_size=BATCH_SIZES[fold]*4)
    ct_test = count_data_items(files_test); STEPS = TTA * ct_test/BATCH_SIZES[fold]/4/REPLICAS
    pred = model.predict(ds_test,steps=STEPS,verbose=VERBOSE)[:TTA*ct_test,] 
    preds[:,0] += np.mean(pred.reshape((ct_test,TTA),order='F'),axis=1) * WGTS[fold]
    
    # REPORT RESULTS
    ### BUG - YG
    list_length = min(len(oof_tar[-1]),len(oof_pred[-1]))
    
    auc = roc_auc_score(oof_tar[-1][:list_length],oof_pred[-1][:list_length])
    oof_val.append(np.max( history.history['val_auc'] ))
    print('#### FOLD %i OOF AUC without TTA = %.3f, with TTA = %.3f'%(fold+1,oof_val[-1],auc))
    
    # PLOT TRAINING
    if DISPLAY_PLOT:
        plt.figure(figsize=(15,5))
        plt.plot(np.arange(EPOCHS[fold]),history.history['auc'],'-o',label='Train AUC',color='#ff7f0e')
        plt.plot(np.arange(EPOCHS[fold]),history.history['val_auc'],'-o',label='Val AUC',color='#1f77b4')
        x = np.argmax( history.history['val_auc'] ); y = np.max( history.history['val_auc'] )
        xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
        plt.scatter(x,y,s=200,color='#1f77b4'); plt.text(x-0.03*xdist,y-0.13*ydist,'max auc\n%.2f'%y,size=14)
        plt.ylabel('AUC',size=14); plt.xlabel('Epoch',size=14)
        plt.legend(loc=2)
        plt2 = plt.gca().twinx()
        plt2.plot(np.arange(EPOCHS[fold]),history.history['loss'],'-o',label='Train Loss',color='#2ca02c')
        plt2.plot(np.arange(EPOCHS[fold]),history.history['val_loss'],'-o',label='Val Loss',color='#d62728')
        x = np.argmin( history.history['val_loss'] ); y = np.min( history.history['val_loss'] )
        ydist = plt.ylim()[1] - plt.ylim()[0]
        plt.scatter(x,y,s=200,color='#d62728'); plt.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
        plt.ylabel('Loss',size=14)
# YG        
#        plt.title('FOLD %i - Image Size %i, EfficientNet B%i, inc2019=%i, inc2018=%i'%
#                (fold+1,IMG_SIZES[fold],EFF_NETS[fold],INC2019[fold],INC2018[fold]),size=18)
#        plt.title('FOLD %i - Image Size %i, %s, inc2019=%i, inc2018=%i'%
#                (fold+1,IMG_SIZES[fold],model_name,INC2019[fold],INC2018[fold]),size=18)
        plt.title('FOLD %i - Image Size %i, %s, Device=%s, TTA=%i'%
                (fold+1,IMG_SIZES[fold],model_name,DEVICE,TTA),size=18)


        plt.legend(loc=3)
        plt.savefig('AUC_' + DEVICE + '_model' + str(MODEL) + '_' + str(SIZE) + '_fold' + str(fold) + '.png' ,bbox_inches='tight', dpi=300)
        plt.show()  

## Calculate OOF AUC
The OOF (out of fold) predictions are saved to disk. If you wish to ensemble multiple models, use the OOF to determine what are the best weights to blend your models with. Choose weights that maximize OOF CV score when used to blend OOF. Then use those same weights to blend your test predictions.

In [ ]:
# COMPUTE OVERALL OOF AUC
oof = np.concatenate(oof_pred); true = np.concatenate(oof_tar);
names = np.concatenate(oof_names); folds = np.concatenate(oof_folds)

### BUG - YG
list_length = min(len(true),len(oof))

auc = roc_auc_score(true[:list_length],oof[:list_length])
print('Overall OOF AUC with TTA = %.3f'%auc)

# SAVE OOF TO DISK
df_oof = pd.DataFrame(dict(
    image_name = names, target=true[:list_length], pred = oof[:list_length], fold=folds))
df_oof.to_csv(DEVICE + '_' + str(MODEL) + '_oof.csv',index=False)
df_oof.tail()

# Step 5: Post process
## Measure prediction times, AUC, model file size

### With TTA

In [ ]:
import time
import statistics

VERBOSE = 1

In [ ]:
%%time

def AUC_time(print_verbose=False, augment=True, aug_lossless = True, TTA=1):

    predict_time_list = []
    AUC_list = []

    oof_pred = []; oof_tar = []

    for fold,(idxT,idxV) in enumerate(skf.split(np.arange(15))):
        if print_verbose: print('Fold %i. Loading the current fold model...'%fold)
        model.load_weights(DEVICE + '_' + str(MODEL) + '_' + str(SIZE) + '_fold-%i.h5'%fold)
        #model.load_weights(DEVICE + '_fold-%i.h5'%fold)

        # PREDICT with TTA # augment=True, aug_lossless = True,
        ds_valid = get_dataset(files_valid,labeled=False,return_image_names=False,
                           augment=True, aug_lossless = True,
                           repeat=True,shuffle=False,dim=SPECIFIC_SIZE,
                           #dim=IMG_SIZES[fold],
                           batch_size=BATCH_SIZES[fold]*4)
        ct_valid = count_data_items(files_valid); STEPS = TTA * ct_valid/BATCH_SIZES[fold]/4/REPLICAS
        if print_verbose: print('Predicting Test with TTA_LL for %i files...'%(ct_valid))     
    
        # Start timer
        ts_eval = time.time()
        pred = model.predict(ds_valid,steps=STEPS,verbose=VERBOSE)[:TTA*ct_valid,] 
        # End timer
        te_eval = time.time()
    
        test_time = (te_eval-ts_eval)/ct_valid
        predict_time_list.append(test_time)
        if print_verbose: print('Fold %i, test_time=%.6f seconds.'%(fold,test_time))
    
        # Add predictions to list
        oof_pred.append( np.mean(pred.reshape((ct_valid,TTA),order='F'),axis=1) )     
        # GET OOF TARGETS AND NAMES
        ds_valid = get_dataset(files_valid, augment=False, aug_lossless = True, repeat=False,dim=SPECIFIC_SIZE,
                           #dim=IMG_SIZES[fold],
            labeled=True, return_image_names=True)
        # Add targets to list
        oof_tar.append( np.array([target.numpy() for img, target in iter(ds_valid.unbatch())]) )    
        # Calculate AUC
        auc = roc_auc_score(oof_tar[-1],oof_pred[-1])
        # Add AUC to list
        AUC_list.append(auc)
        if print_verbose: print('Fold %i, AUC=%.6f'%(fold,auc))

    # Calculate AUC mean value to list 
    AUC_mean = statistics.mean(AUC_list) # mean
    # Calculate AUC standard deviation value to list
    AUC_std = statistics.stdev(AUC_list) # standard devition
    print('#### TTA = %d'%TTA)
    print('#### OOF AUC with TTA_LL: mean = %.6f, stdev = %.6f.'%(AUC_mean,AUC_std))

    # Add AUC mean value to list 
    predict_time_mean = statistics.mean(predict_time_list) # mean
    # Add AUC standard deviation value to list
    predict_time_std = statistics.stdev(predict_time_list) # standard devition
    print('#### Time without TTA_LL: mean = %.6f, stdev = %.6f seconds.'%(predict_time_mean,predict_time_std)) 
    
    return AUC_mean, AUC_std, predict_time_mean, predict_time_std

#### LOSSLESS TTA - withTTA_LL

In [ ]:
%%time

# _withTTA_LL_

VERBOSE = 0

TTA_list = [1,2,4,8,16,32,64,128]

withTTA_LL_AUC_mean_list = []
withTTA_LL_AUC_std_list = []

withTTA_LL_time_mean_list = []
withTTA_LL_time_std_list = []

for i in tqdm(TTA_list):
    AUC_mean, AUC_std, predict_time_mean, predict_time_std = AUC_time(print_verbose=False,augment=True, aug_lossless = True, TTA=i)
    
    withTTA_LL_AUC_mean_list.append(AUC_mean)
    withTTA_LL_AUC_std_list.append(AUC_std) 
    withTTA_LL_time_mean_list.append(predict_time_mean)
    withTTA_LL_time_std_list.append(predict_time_std)

In [ ]:
# PLOT: AUC - withTTA_LL
plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(10,5))
#plt.plot(TTA_list,withTTA_LL_AUC_mean_list,'-o',label='AUC mean',color='#ff7f0e')
plt.errorbar(np.log2(TTA_list),withTTA_LL_AUC_mean_list,yerr=withTTA_LL_AUC_std_list, fmt='-o', label='AUC mean',color='#1f77b4');
#plt.plot(TTA_list,withTTA_LL_AUC_std_list,'-o',label='AUC std',color='#1f77b4')
x = np.argmax(withTTA_LL_AUC_mean_list); y = np.max(withTTA_LL_AUC_mean_list)
xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
plt.scatter(x,y,s=200,color='red'); plt.text(x-0.03*xdist,y-0.13*ydist,'max auc\n%.3f'%y,size=14,color='red')
plt.ylabel('AUC',size=14); plt.xlabel('log2(TTA steps)',size=14)
plt.title('withTTA_LL, Image Size=%i, %s, Device=%s, Epochs=%i'%
                (IMG_SIZES[fold],model_name,DEVICE,EPOCH),size=14)
#plt.legend(loc='best')
plt.savefig('AUC_' + 'withTTA_LL' + DEVICE + '_' + str(MODEL) + '_' + str(SIZE) + '_ep' + str(EPOCH) + '.png' ,bbox_inches='tight', dpi=300)
plt.show()  

In [ ]:
# PLOT: PREDICTION TIME - withTTA_LL
plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(10,5))
#plt.plot(TTA_list,withTTA_LL_AUC_mean_list,'-o',label='AUC mean',color='#ff7f0e')
plt.errorbar(TTA_list,withTTA_LL_time_mean_list,yerr=withTTA_LL_time_std_list, fmt='-o', label='lossless',color='#1f77b4');
#plt.plot(TTA_list,withTTA_LL_AUC_std_list,'-o',label='AUC std',color='#1f77b4')
x = np.argmin(withTTA_LL_time_mean_list); y = np.min(withTTA_LL_time_mean_list)
xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
plt.scatter(x,y,s=200,color='red'); plt.text(x-0.03*xdist,y-0.13*ydist,'min time\n%.3f'%y,size=14,color='red')
plt.ylabel('Prediction Time',size=14); plt.xlabel('TTA steps',size=14)
plt.title('withTTA_LL, Image Size=%i, %s, Device=%s, Epochs=%i'%
                (IMG_SIZES[fold],model_name,DEVICE,EPOCH),size=14)
#plt.legend(loc='best')
plt.savefig('TIME_' + 'withTTA_LL' + '_' + DEVICE + '_' + str(MODEL) + '_' + str(SIZE) + '_ep' + str(EPOCH) + '.png' ,bbox_inches='tight', dpi=300)
plt.show()  

#### FULL TTA - withTTA

In [ ]:
%%time

# withTTA_

VERBOSE = 0

TTA_list = [1,2,4,8,16,32,64,128]

withTTA_AUC_mean_list = []
withTTA_AUC_std_list = []

withTTA_time_mean_list = []
withTTA_time_std_list = []

for i in tqdm(TTA_list):
    AUC_mean, AUC_std, predict_time_mean, predict_time_std = AUC_time(print_verbose=False, augment=True, aug_lossless = False, TTA=i)
    
    withTTA_AUC_mean_list.append(AUC_mean)
    withTTA_AUC_std_list.append(AUC_std) 
    withTTA_time_mean_list.append(predict_time_mean)
    withTTA_time_std_list.append(predict_time_std)

In [ ]:
# PLOT: AUC - withTTA_LL
plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(10,5))
#plt.plot(TTA_list,withTTA_LL_AUC_mean_list,'-o',label='AUC mean',color='#ff7f0e')
plt.errorbar(np.log2(TTA_list),withTTA_AUC_mean_list,yerr=withTTA_AUC_std_list, fmt='-o', label='full',color='#1f77b4');
#plt.plot(TTA_list,withTTA_LL_AUC_std_list,'-o',label='AUC std',color='#1f77b4')
x = np.argmax(withTTA_AUC_mean_list); y = np.max(withTTA_AUC_mean_list)
xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
plt.scatter(x,y,s=200,color='red'); plt.text(x-0.03*xdist,y-0.13*ydist,'max auc\n%.3f'%y,size=14,color='red')
plt.ylabel('AUC',size=14); plt.xlabel('log2(TTA steps)',size=14)
plt.title('withTTA_full, Image Size=%i, %s, Device=%s, Epochs=%i'%
                (IMG_SIZES[fold],model_name,DEVICE,EPOCH),size=14)
#plt.legend(loc='best')
plt.savefig('AUC_' + 'withTTA_full' + '_' + DEVICE + '_' + str(MODEL) + '_' + str(SIZE) + '_ep' + str(EPOCH) + '.png' ,bbox_inches='tight', dpi=300)
plt.show()  

In [ ]:
# PLOT: PREDICTION TIME - withTTA_LL
plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(10,5))
#plt.plot(TTA_list,withTTA_LL_AUC_mean_list,'-o',label='AUC mean',color='#ff7f0e')
plt.errorbar(TTA_list,withTTA_time_mean_list,yerr=withTTA_time_std_list, fmt='-o', label='full',color='#1f77b4');
#plt.plot(TTA_list,withTTA_LL_AUC_std_list,'-o',label='AUC std',color='#1f77b4')
x = np.argmin(withTTA_LL_time_mean_list); y = np.min(withTTA_LL_time_mean_list)
xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
plt.scatter(x,y,s=200,color='red'); plt.text(x-0.03*xdist,y-0.13*ydist,'min time\n%.3f'%y,size=14,color='red')
plt.ylabel('Prediction Time',size=14); plt.xlabel('TTA steps',size=14)
plt.title('withTTA, Image Size=%i, %s, Device=%s, Epochs=%i'%
                (IMG_SIZES[fold],model_name,DEVICE,EPOCH),size=14)
#plt.legend(loc='best')
plt.savefig('TIME_' + 'withTTA_full' + '_' + DEVICE + '_' + str(MODEL) + '_' + str(SIZE) + '_ep' + str(EPOCH) + '.png' ,bbox_inches='tight', dpi=300)
plt.show()  

#### Both TTAs on the same plot - AUC and TIME

In [ ]:
# PLOT: AUC - withTTA_LL - lossless
plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(10,5))
#plt.plot(TTA_list,withTTA_LL_AUC_mean_list,'-o',label='AUC mean',color='#ff7f0e')
plt.errorbar(np.log2(TTA_list),withTTA_LL_AUC_mean_list,yerr=withTTA_LL_AUC_std_list, fmt='-o', label='lossless',color='blue');
#plt.plot(TTA_list,withTTA_LL_AUC_std_list,'-o',label='AUC std',color='#1f77b4')
x = np.argmax(withTTA_LL_AUC_mean_list); y = np.max(withTTA_LL_AUC_mean_list)

withTTA_LL_AUC_mean_max = np.max(withTTA_LL_AUC_mean_list)
withTTA_LL_AUC_std_max = withTTA_LL_AUC_std_list[np.argmax(withTTA_LL_AUC_mean_list)]
withTTA_LL_AUC_mean_TTA = np.argmax(withTTA_LL_AUC_mean_list)
print('i=%d,'%withTTA_LL_AUC_mean_TTA, 'withTTA_LL_AUC_max=%.3f'%withTTA_LL_AUC_mean_max, '(+-%.3f)'%withTTA_LL_AUC_std_max)

xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
plt.scatter(x,y,s=200,color='red'); plt.text(x-0.03*xdist,y-0.13*ydist,'max auc\n%.3f'%y,size=14,color='blue')

# PLOT: AUC - withTTA - full
plt.errorbar(np.log2(TTA_list),withTTA_AUC_mean_list,yerr=withTTA_AUC_std_list, fmt='-o', label='full',color='green');
#plt.plot(TTA_list,withTTA_LL_AUC_std_list,'-o',label='AUC std',color='#1f77b4')
x = np.argmax(withTTA_AUC_mean_list); y = np.max(withTTA_AUC_mean_list)

withTTA_AUC_mean_max = np.max(withTTA_AUC_mean_list)
withTTA_AUC_std_max = withTTA_AUC_std_list[np.argmax(withTTA_AUC_mean_list)]
withTTA_AUC_mean_TTA = np.argmax(withTTA_AUC_mean_list)
print('i=%d,'%withTTA_AUC_mean_TTA, 'withTTA_AUC_max=%.3f'%withTTA_AUC_mean_max, '(+-%.3f)'%withTTA_AUC_std_max)

xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
plt.scatter(x,y,s=200,color='red'); plt.text(x-0.03*xdist,y-0.13*ydist,'max auc\n%.3f'%y,size=14,color='green')

plt.ylabel('AUC',size=14); plt.xlabel('log2(TTA steps)',size=14)
plt.title('Image Size=%i, %s, Device=%s, Epochs=%i'%
                (IMG_SIZES[fold],model_name,DEVICE,EPOCH),size=14)
plt.legend(loc='best')
plt.savefig('AUC_both_TTA_' + DEVICE + '_' + str(MODEL) + '_' + str(SIZE) + '_ep' + str(EPOCH) + '.png' ,bbox_inches='tight', dpi=300)
plt.show()  

In [ ]:
# PLOT: PREDICTION TIME - withTTA_LL - lossless
plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(10,5))
#plt.plot(TTA_list,withTTA_LL_AUC_mean_list,'-o',label='AUC mean',color='#ff7f0e')
plt.errorbar(TTA_list,withTTA_LL_time_mean_list,yerr=withTTA_LL_time_std_list, fmt='-o', label='lossless',color='blue');
#plt.plot(TTA_list,withTTA_LL_AUC_std_list,'-o',label='AUC std',color='#1f77b4')
x = np.argmin(withTTA_LL_time_mean_list); y = np.min(withTTA_LL_time_mean_list)

withTTA_LL_time_mean_max = withTTA_LL_time_mean_list[np.argmax(withTTA_LL_AUC_mean_list)]
withTTA_LL_time_std_max = withTTA_LL_time_std_list[np.argmax(withTTA_LL_AUC_mean_list)]
withTTA_LL_time_mean_TTA = np.argmax(withTTA_LL_AUC_mean_list)
print('i=%d,'%withTTA_LL_time_mean_TTA, 'withTTA_LL_time_mean_max=%.3f'%withTTA_LL_time_mean_max, '(+-%.3f)'%withTTA_LL_time_std_max)

xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
plt.scatter(x,y,s=200,color='red'); plt.text(x-0.03*xdist,y-0.13*ydist,'min time\n%.3f'%y,size=14,color='blue')

# PLOT: PREDICTION TIME - withTTA - full
plt.errorbar(TTA_list,withTTA_time_mean_list,yerr=withTTA_time_std_list, fmt='-o', label='full',color='green');
#plt.plot(TTA_list,withTTA_LL_AUC_std_list,'-o',label='AUC std',color='#1f77b4')
x = np.argmin(withTTA_time_mean_list); y = np.min(withTTA_time_mean_list)

withTTA_time_mean_max = withTTA_time_mean_list[np.argmax(withTTA_AUC_mean_list)]
withTTA_time_std_max = withTTA_time_std_list[np.argmax(withTTA_AUC_mean_list)]
withTTA_time_mean_TTA = np.argmax(withTTA_AUC_mean_list)
print('i=%d,'%withTTA_time_mean_TTA, 'withTTA_time_mean_max=%.3f'%withTTA_time_mean_max, '(+-%.3f)'%withTTA_time_std_max)

xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
plt.scatter(x,y,s=200,color='red'); plt.text(x-0.03*xdist,y+0.1*ydist,'min time\n%.3f'%y,size=14,color='green')

plt.ylabel('Prediction Time',size=14); plt.xlabel('TTA steps',size=14)
plt.title('withTTA_LL, Image Size=%i, %s, Device=%s, Epochs=%i'%
                (IMG_SIZES[fold],model_name,DEVICE,EPOCH),size=14)
plt.legend(loc='best')
plt.savefig('TIME_both_TTA_' + DEVICE + '_' + str(MODEL) + '_' + str(SIZE) + '_ep' + str(EPOCH) + '.png' ,bbox_inches='tight', dpi=300)
plt.show()  

### Without TTA

In [ ]:
%%time

TTA = 1
predict_woTTA_time_list = []
AUC_woTTA_list = []

oof_pred = []; oof_tar = []

for fold,(idxT,idxV) in enumerate(skf.split(np.arange(15))):
    print('Fold %i. Loading the current fold model...'%fold)
    model.load_weights(DEVICE + '_' + str(MODEL) + '_' + str(SIZE) + '_fold-%i.h5'%fold)
    #model.load_weights(DEVICE + '_fold-%i.h5'%fold)

    # PREDICT without TTA # augment=False
    ds_valid = get_dataset(files_valid,labeled=False,return_image_names=False,augment=False,
            repeat=True,shuffle=False,dim=SPECIFIC_SIZE,
                           #dim=IMG_SIZES[fold],
                           batch_size=BATCH_SIZES[fold]*4)
    ct_valid = count_data_items(files_valid); STEPS = TTA * ct_valid/BATCH_SIZES[fold]/4/REPLICAS
    print('Predicting Test without TTA for %i files...'%(ct_valid))     
    
    # Start timer
    ts_eval = time.time()
    pred = model.predict(ds_valid,steps=STEPS,verbose=VERBOSE)[:TTA*ct_valid,] 
    # End timer
    te_eval = time.time()
    
    test_time = (te_eval-ts_eval)/ct_valid
    predict_woTTA_time_list.append(test_time)
    print('Fold %i, test_time=%.6f seconds.'%(fold,test_time))
    
    # Add predictions to list
    oof_pred.append( np.mean(pred.reshape((ct_valid,TTA),order='F'),axis=1) )     
    # GET OOF TARGETS AND NAMES
    ds_valid = get_dataset(files_valid, augment=False, repeat=False,dim=SPECIFIC_SIZE,
                           #dim=IMG_SIZES[fold],
            labeled=True, return_image_names=True)
    # Add targets to list
    oof_tar.append( np.array([target.numpy() for img, target in iter(ds_valid.unbatch())]) )    
    # Calculate AUC
    auc = roc_auc_score(oof_tar[-1],oof_pred[-1])
    # Add AUC to list
    AUC_woTTA_list.append(auc)
    print('Fold %i, AUC=%.6f'%(fold,auc))

# Calculate AUC mean value to list 
AUC_woTTA_mean = statistics.mean(AUC_woTTA_list) # mean
# Calculate AUC standard deviation value to list
AUC_woTTA_std = statistics.stdev(AUC_woTTA_list) # standard devition
print('#### OOF AUC without TTA: mean = %.6f, stdev = %.6f.'%(AUC_woTTA_mean,AUC_woTTA_std))

# Add AUC mean value to list 
predict_woTTA_time_mean = statistics.mean(predict_woTTA_time_list) # mean
# Add AUC standard deviation value to list
predict_woTTA_time_std = statistics.stdev(predict_woTTA_time_list) # standard devition
print('#### Time without TTA: mean = %.6f, stdev = %.6f seconds.'%(predict_woTTA_time_mean,predict_woTTA_time_std))

### Measure model file size

In [ ]:
import os
model_size = os.path.getsize(DEVICE + '_' + str(MODEL) + '_' + str(SIZE) + '_fold-0.h5') # >> 20
print(str(model_size) + ' Bytes')

### Save to file

In [ ]:
results = pd.DataFrame(data=[[MODEL, model_size, TRAINABLE, AUGMENTATION, AUG_LOSSLESS,
                              EPOCHS[0], SIZE,
                              withTTA_LL_time_mean_max, withTTA_LL_time_std_max, withTTA_LL_AUC_mean_max, withTTA_LL_AUC_std_max,
                              withTTA_time_mean_max, withTTA_time_std_max, withTTA_AUC_mean_max, withTTA_AUC_std_max,
                              predict_woTTA_time_mean, predict_woTTA_time_std, AUC_woTTA_mean, AUC_woTTA_std]],
                       columns=['model', 'model_size', 'trainable', 'augmentation', 'aug_lossless',
                                'epochs', 'image_size',
                                'TTA_LL_time_mean','TTA_LL_time_std','TTA_LL_AUC_mean','TTA_LL_AUC_std',
                                'TTA_time_mean','TTA_time_std','TTA_AUC_mean','TTA_AUC_std',
                                'woTTA_time_mean','woTTA_time_std', 'woTTA_AUC_mean', 'woTTA_AUC_std'])
experiment_title = DEVICE + '_' + str(MODEL) + '_s' + str(SIZE) + '_ep' + str(EPOCHS[0]) + '_train' + str(TRAINABLE) + '_aug' + str(AUGMENTATION) + '_loss' + str(AUG_LOSSLESS) + '_time_AUC'
results.to_csv(experiment_title + '.csv', index=False)
results.head()

In [ ]:
!ls -all

In [ ]:
from zipfile import ZipFile
from os.path import basename

# Zip the files from given directory that matches the filter
def zipFilesInDir(dirName, zipFileName, filter):
   # create a ZipFile object
   with ZipFile(zipFileName, 'w') as zipObj:
       # Iterate over all the files in directory
       for folderName, subfolders, filenames in os.walk(dirName):
           for filename in filenames:
               if filter(filename):
                   # create complete filepath of file in directory
                   filePath = os.path.join(folderName, filename)
                   # Add file to zip
                   zipObj.write(filePath, basename(filePath))
print('*** Create a zip archive of *.png *.ipynb *.csv files form a directory ***')
zipFilesInDir('.', experiment_title+'_csv_.zip', lambda name : 'csv' in name)
zipFilesInDir('.', experiment_title+'_ipynb_.zip', lambda name : 'ipynb' in name)
zipFilesInDir('.', experiment_title+'_png_.zip', lambda name : 'png' in name)
zipFilesInDir('.', experiment_title+'_all.zip', lambda name : '_.zip' in name)

In [ ]:
! ls -all *.zip